# Notebook Explicativo: Análise e Modelagem de Dados

## 1. Introdução
Este notebook demonstra um fluxo completo de análise de dados, desde o carregamento e pré-processamento até a modelagem preditiva usando regressão linear. Vamos trabalhar com três datasets diferentes, aplicar transformações necessárias e avaliar o desempenho dos modelos.

## 2. Importar Bibliotecas

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

**Explicação:** Importamos as bibliotecas essenciais para:
- `os`: manipulação de caminhos de arquivos
- `pandas`: manipulação e análise de dados
- `matplotlib`: visualização de gráficos
- `scikit-learn`: machine learning (modelo, divisão de dados, métricas)

## 3. Funções Auxiliares

### Função: `preprocess_temperature`

In [ ]:
def preprocess_temperature(df, temp_col, unit_col):
    def convert(row):
        if unit_col and row.get(unit_col, "").lower() in ['f', 'fahrenheit']:
            return (row[temp_col] - 32) * 5/9
        return row[temp_col]
    df[temp_col] = df.apply(convert, axis=1)
    return df

**Explicação:** Esta função converte temperaturas de Fahrenheit para Celsius quando necessário. Ela verifica a coluna de unidade e aplica a fórmula de conversão apenas para valores em Fahrenheit.

### Função: `load_dataset`

In [ ]:
def load_dataset(path, conversions=None):
    df = pd.read_csv(path, low_memory=False)
    if conversions:
        for temp_col, unit_col in conversions:
            if unit_col and unit_col in df.columns:
                df = preprocess_temperature(df, temp_col, unit_col)
    return df

**Explicação:** Carrega um dataset CSV e aplica conversões de temperatura especificadas. O parâmetro `low_memory=False` evita problemas com tipos de dados mistos.

### Função: `train_and_evaluate`

In [ ]:
def train_and_evaluate(df, feature_col, target_col, dataset_name, save_path=None):
    # Remove linhas com valores missing nas colunas relevantes
    df = df.dropna(subset=[feature_col, target_col])
    
    # Prepara features (X) e target (y)
    X = df[[feature_col]].values
    y = df[target_col].values

    # Divide os dados em treino (70%) e teste (30%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Cria e treina o modelo de regressão linear
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Calcula métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Exibe resultados
    print(f"\n=== Resultados para {dataset_name} ===")
    print(f"MAE: {mae:.3f}")
    print(f"MSE: {mse:.3f}")
    print(f"R² : {r2:.3f}")

    # Cria gráfico de dispersão comparando valores reais vs previstos
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.xlabel("Valores Reais")
    plt.ylabel("Valores Previstos")
    plt.title(f"Reais vs Previstos - {dataset_name}")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

    # Salva o gráfico se um caminho for fornecido
    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

    return {"MAE": mae, "MSE": mse, "R2": r2}

**Explicação:** Esta função completa executa todo o pipeline de modelagem:
1. **Limpeza**: Remove valores missing
2. **Preparação**: Separa features e target
3. **Divisão**: Separa dados em treino e teste
4. **Treinamento**: Ajusta o modelo de regressão linear
5. **Avaliação**: Calcula métricas de performance
6. **Visualização**: Gera gráfico comparativo

## 4. Configuração de Caminhos

In [ ]:
DATA_PATH = "data"
RESULTS_PATH = os.path.join("results", "graficos")
os.makedirs(RESULTS_PATH, exist_ok=True)

**Explicação:** Configura os diretórios para dados e resultados. `os.makedirs(..., exist_ok=True)` garante que as pastas sejam criadas se não existirem.

## 5. Execução por Dataset

### Dataset 1: Umidade vs Temperatura

In [ ]:
df1 = load_dataset(
    os.path.join(DATA_PATH, "ds_hum_vs_temp_dirty.csv"), 
    conversions=[("temperatura", "unidade_temperatura")]
)
metrics1 = train_and_evaluate(
    df1, "temperatura", "humidade", "Umidade vs Temperatura", 
    save_path=os.path.join(RESULTS_PATH, "umidade_vs_temp.png")
)

**Explicação:** 
- **Objetivo**: Prever umidade com base na temperatura
- **Pré-processamento**: Converte temperatura se estiver em Fahrenheit
- **Feature**: temperatura
- **Target**: humidade

### Dataset 2: MinTemp vs MaxTemp

In [ ]:
df2 = load_dataset(
    os.path.join(DATA_PATH, "ds_min_temp_vs_max_temp_raw.csv"), 
    conversions=[("MinTemp", None), ("MaxTemp", None)]
)
metrics2 = train_and_evaluate(
    df2, "MinTemp", "MaxTemp", "MinTemp vs MaxTemp", 
    save_path=os.path.join(RESULTS_PATH, "min_vs_max.png")
)

**Explicação:** 
- **Objetivo**: Prever temperatura máxima com base na temperatura mínima
- **Pré-processamento**: Sem conversão de unidades (None)
- **Feature**: MinTemp (temperatura mínima)
- **Target**: MaxTemp (temperatura máxima)

### Dataset 3: Salinity vs Temperatura

In [ ]:
df3 = load_dataset(
    os.path.join(DATA_PATH, "ds_salinity_vs_temp_raw.csv"), 
    conversions=[("T_degC", None)]
)
metrics3 = train_and_evaluate(df3, "Salnty", "T_degC", "Salinity vs Temperatura")

**Explicação:** 
- **Objetivo**: Prever temperatura da água com base na salinidade
- **Feature**: Salnty (salinidade)
- **Target**: T_degC (temperatura em Celsius)

## 6. Salvar Métricas

In [ ]:
with open(os.path.join("results", "metrics.txt"), "w") as f:
    f.write("Resultados dos Modelos:\n")
    f.write(f"Dataset 1: {metrics1}\n")
    f.write(f"Dataset 2: {metrics2}\n")
    f.write(f"Dataset 3: {metrics3}\n")

**Explicação:** Salva todas as métricas em um arquivo texto para documentação e análise posterior.